In [ ]:
%run ./creating_arrays.ipynb

In [ ]:
from pathlib import Path
import os
import re
import shutil
import talib
import datetime
import glob
import zipfile
import pandas as pd
from pandas.api.types import is_numeric_dtype
import inspect
import talib
import time
import numpy as np
import requests
import urllib.request

In [ ]:
def download_monthly_data(month_array, symbol, chart_time):
    #downloading monthly data
    root_dir = Path.cwd()
    # Create the new folder path
    folder_path = Path(
        download_dir) / f"{symbol}-{chart_time}-monthly_data"
    folder_path.mkdir(parents=True, exist_ok=True)
    count = 0
    for month in month_array:
        # Construct the link
        link = f"{BINANCE_MONTHLY_URL}{symbol}/{chart_time}/{symbol}-{chart_time}-{month}.zip"
        symbol_object = f"{symbol}-{chart_time}-{month}.zip"
        # Create the file path
        file_path = Path(folder_path) / symbol_object
        if not file_path.exists():
            try:
                # Download the file
                urllib.request.urlretrieve(link, file_path)
                count += 1
            except:
                #                     print(f'{link} not found')
                continue
    if count > 0:
        print (f"Monthly Data Downloaded for {symbol},{chart_time}")
    else:
        print (f"you're already up to date for monthly data for {symbol},{chart_time}")
    

    
def download_daily_data(day_array, symbol, chart_time):
    #downloading daily data
    root_dir = Path.cwd()
    # Create the new folder path
    folder_path = Path(
        download_dir) / f"{symbol}-{chart_time}-daily_data"
    folder_path.mkdir(parents=True, exist_ok=True)
    count = 0
    for day in day_array:
        # Construct the link
        link = f"{BINANCE_DAILY_URL}{symbol}/{chart_time}/{symbol}-{chart_time}-{day}.zip"
        symbol_object = f"{symbol}-{chart_time}-{day}.zip"
        # Create the file path
        file_path = Path(folder_path) / symbol_object
        if not file_path.exists():
            try:
                # Download the file
                urllib.request.urlretrieve(link, file_path)
                count += 1
            except:
                #                     print(f'{link} not found')
                continue
    if count > 0:
        print(f"Daily Data Downloaded for {symbol},{chart_time}")
    else:
        print(f"you're already up to date for daily data for {symbol},{chart_time}")


In [ ]:
def download_data_and_concatenate(
        master_dictionary,
        win_perc=master_dictionary["win_percentage"],
        loss_perc=master_dictionary["loss_percentage"]):
    # Set the start date and end date
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2080, 1, 1)

    # Initialize the list of year-month pairs
    MONTH_ARRAY = []

    # Iterate over the dates from start to end
    date = start_date
    while date < end_date:
        # Format the date as a year-month pair
        year_month = date.strftime("%Y-%m")
        #
        # Add the year-month pair to the list
        MONTH_ARRAY.append(year_month)

        # Increment the date by one month
        date += timedelta(days=31)
    # Get the current date
    now = datetime.now()

    # Format the date as a string in the desired format
    date_string = now.strftime("%Y-%m")

    idx = MONTH_ARRAY.index(date_string)
    MONTH_ARRAY = MONTH_ARRAY[:idx + 1]

    # Get the current date
    now = datetime.now()

    # Get the first day of the current month
    first_day = now.replace(day=1)

    # Create an empty list to store the days
    DAY_ARRAY = []

    # Loop through the days from the first day of the current month to today
    while first_day <= now:
        # Format the date as a string in the desired format
        date_string = first_day.strftime("%Y-%m-%d")

        # Add the date string to the list
        DAY_ARRAY.append(date_string)

        # Move to the next day
        first_day += timedelta(days=1)
   

    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            print(f"setting up things for {symbol},{chart_time}")

            # Set up an empty list for the data frames
            df_list = []

            # Compile the regular expression pattern
            pattern = re.compile(
                f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

            # Compile the regular expression pattern for daily zip files
            pattern_daily = re.compile(
                f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}-\d{{2}}\.zip$")

            # Create the new folder path for daily ZIP files
            new_daily_zip_folder_path = os.path.join(
                download_dir, f"{symbol}-{chart_time}-daily_data")

            # Create the new folder path for ZIP files
            new_zip_folder_path = os.path.join(
                download_dir, f"{symbol}-{chart_time}-monthly_data")

            # Create the new folder path for CSV files
            new_csv_folder_path = os.path.join(output_dir,
                                               f"{symbol}-{chart_time}")

            # Set the file name
            concatenated_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"

            # Construct the file path
            concatenated_file_path = os.path.join(new_csv_folder_path,
                                                  concatenated_file_name)

            if not Path(concatenated_file_path).exists():
                print(
                    f"setting up things for the new symbol-chart_time combination : {symbol}{chart_time}"
                )
                download_monthly_data(month_array=MONTH_ARRAY,
                                      symbol=symbol,
                                      chart_time=chart_time)
                download_daily_data(day_array=DAY_ARRAY,
                                    symbol=symbol,
                                    chart_time=chart_time)

                # Iterate over the files in the new zip folder
                for file in os.listdir(new_zip_folder_path):
                    # Check if the file matches the pattern
                    if pattern.match(file):
                        # Construct the file path
                        file_path = os.path.join(new_zip_folder_path, file)

                        # Extract the ZIP file
                        with zipfile.ZipFile(file_path, "r") as zip_ref:
                            zip_ref.extractall(new_csv_folder_path)

                        # Construct the CSV file path
                        csv_file_path = os.path.join(
                            new_csv_folder_path,
                            f"{symbol}-{chart_time}{file[-12:-4]}.csv")

                        # Read the CSV file into a data frame, ignoring the headers
                        df = pd.read_csv(csv_file_path, header=None)

                        # Remove the first row (which contains the header)
                        df = df.iloc[1:]

                        # Add it to the list
                        df_list.append(df)

                # Iterate over the files in the new daily zip folder
                for file in os.listdir(new_daily_zip_folder_path):
                    # Check if the file matches the pattern
                    if pattern_daily.match(file):
                        # Construct the file path
                        file_path = os.path.join(new_daily_zip_folder_path,
                                                 file)

                        # Extract the ZIP file
                        with zipfile.ZipFile(file_path, "r") as zip_ref:
                            zip_ref.extractall(new_csv_folder_path)
                        # Construct the CSV file path
                        csv_file_path = os.path.join(
                            new_csv_folder_path,
                            f"{symbol}-{chart_time}-{file.split('-')[-3]}-{file.split('-')[-2]}-{file.split('-')[-1][:-4]}.csv"
                        )
                        # Read the CSV file into a data frame, ignoring the headers
                        df = pd.read_csv(csv_file_path, header=None)

                        # Remove the first row (which contains the header)
                        df = df.iloc[1:]

                        # Add it to the list
                        df_list.append(df)
                # Concatenate the data frames in the list
                df_final = pd.concat(df_list, ignore_index=True)

                # Read the headers from the first CSV file
                headers = pd.read_csv(csv_file_path, nrows=1).columns

                # Set the headers as the column names of the final dataframe
                df_final.columns = headers

                # Convert 'open_time' and 'close_time' columns to datetime
                df_final['open_time'] = pd.to_datetime(
                    df_final['open_time'], unit='ms').dt.tz_localize(
                        'UTC').dt.tz_convert('Asia/Kolkata')

                # Delete the 'ignore' column
                df_final = df_final.drop(['ignore', 'close_time'], axis=1)

                # Add a new column called 'entry' that will take open of candle
                df_final['entry'] = df_final['open']

                # Write the data frame to the Excel file
                df_final.to_csv(concatenated_file_path, index=False)

                directory_final = Path(
                    concatenated_file_path).parent  # Get the parent directory

                # deleting all the other csvs
                for file_path in directory_final.iterdir():
                    if file_path != Path(concatenated_file_path):
                        if file_path.is_file():
                            file_path.unlink()
                return f"things set up for the new symbol-chart_time combination : {symbol}{chart_time}"
            else:
                print(
                    f"setting up things only for incremental data for {symbol}{chart_time}"
                )
                df_list = []
                df = pd.read_csv(concatenated_file_path, header=None)
                last_record = df.iloc[-1:]
                df_list.append(last_record)

                df = pd.read_csv(concatenated_file_path)
                df['open_time'] = pd.to_datetime(df['open_time'])
                last_record = df.iloc[-1:]

                def calculate_reference_time(chart_time, base_time):
                    # Extract the numeric value and unit from chart_time
                    numeric_value = int(chart_time[:-1])
                    unit = chart_time[-1]

                    # Define a dictionary to map units to timedelta objects
                    time_units = {
                        'm': timedelta(minutes=numeric_value),
                        'h': timedelta(hours=numeric_value),
                    }

                    # Calculate the reference time based on the unit and numeric value
                    reference_time = base_time - time_units[unit]
                    return reference_time.strftime('%H:%M:%S%z')

                base_time = datetime.strptime("05:30:00+0530", '%H:%M:%S%z')
                reference_time = calculate_reference_time(
                    chart_time, base_time)
                print(reference_time)
                # Check if the time part of 'open_time' is "05:15:00+05:30"
                if last_record['open_time'].dt.strftime(
                        '%H:%M:%S%z').item() == reference_time:
                    # Calculate the last processed date by subtracting 1 day from the date part
                    last_processed_date = (
                        last_record['open_time'].dt.date -
                        pd.DateOffset(days=1)).item().strftime('%Y-%m-%d')
                    last_processed_date = datetime.strptime(
                        last_processed_date, "%Y-%m-%d")
                    print(last_processed_date)
                    # Get today's date
                    today = datetime.today()
                    print(today)

                    # Initialize DAY_ARRAY
                    DAY_ARRAY = []
                    # Start from the day after last_processed_date
                    current_day = last_processed_date + timedelta(days=1)

                    while current_day <= today:
                        # Format the date as a string in the desired format
                        date_string = current_day.strftime("%Y-%m-%d")

                        # Add the date string to DAY_ARRAY
                        DAY_ARRAY.append(date_string)

                        # Move to the next day
                        current_day += timedelta(days=1)

                    download_daily_data(day_array=DAY_ARRAY,
                                        symbol=symbol,
                                        chart_time=chart_time)

                    # Iterate over the files in the new daily zip folder
                    for file in Path(new_daily_zip_folder_path).iterdir():
                        # Check if the file matches the pattern
                        if pattern_daily.match(file.name):
                            # Extract the date part from the file name (e.g., "2023-08-19")
                            zip_date = file.name.split('-')[-3:]
                            zip_date_str = '-'.join(zip_date).replace(
                                '.zip', '')

                            # Check if the date is in DAY_ARRAY
                            if zip_date_str in DAY_ARRAY:
                                # Construct the file path
                                file_path = file

                                # Extract the ZIP file
                                with zipfile.ZipFile(file_path,
                                                     "r") as zip_ref:
                                    zip_ref.extractall(new_csv_folder_path)

                                # Construct the CSV file path
                                csv_file_path = Path(
                                    new_csv_folder_path
                                ) / f"{symbol}-{chart_time}-{zip_date_str}.csv"

                                # Read the CSV file into a data frame, ignoring the headers
                                df = pd.read_csv(csv_file_path, header=None)

                                # Remove the first row (which contains the header)
                                df = df.iloc[1:]

                                # Add it to the list
                                df_list.append(df)
                    for df in df_list[1:]:
                        # Convert 'open_time' and 'close_time' columns to datetime
                        df[0] = pd.to_datetime(
                            df[0], unit='ms').dt.tz_localize(
                                'UTC').dt.tz_convert('Asia/Kolkata')
                        cols_to_drop = [6, 11]
                        df.drop(df.columns[cols_to_drop], axis=1, inplace=True)
    # Concatenate the data frames in the list
                    if len(df_list) == 1:
                        return "no need to do anything, you're already having all the latest data"
                    else:
                        df_final = pd.concat(df_list[1:], ignore_index=True)
                        display(df_final.head())
                        # Read the headers from the first CSV file
                        headers = pd.read_csv(concatenated_file_path,
                                              nrows=1).columns

                        # Set the headers as the column names of the final dataframe
                        df_final.columns = headers[0:len(headers) - 1]

                        # Add a new column called 'entry' that will take open of candle
                        df_final['entry'] = df_final['open']
                        display(df_final.head())
                        # Append the data frame to the existing CSV file in append mode without headers
                        df_final.to_csv(concatenated_file_path,
                                        mode='a',
                                        header=False,
                                        index=False)
                        directory_final = Path(
                            concatenated_file_path
                        ).parent  # Get the parent directory

                        # deleting all the other csvs
                        for file_path in directory_final.iterdir():
                            if file_path != Path(concatenated_file_path):
                                if file_path.is_file():
                                    file_path.unlink()
                        return f"things set up only for incremental data for {symbol}{chart_time}"

                else:
                    return f"Something is wrong with data. Last recorded time is not in sync with what is expected. Expected {reference_time} but got {last_record['open_time'].dt.strftime('%H:%M:%S%z').item()}"

In [ ]:
def calculate_wins_losses(master_dictionary, win_perc=0.73, loss_perc=0.4):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            print(f"calculating for {symbol}{chart_time}")
            # Construct the file name
            file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
            file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"
            # Read the CSV file into a dataframe
            df = pd.read_csv(file_path)
            df["if_short"] = 0
            df["if_long"] = 0
            df["long_target"] = np.nan
            df["short_target"] = np.nan
            df["long_stop_loss"] = np.nan
            df["short_stop_loss"] = np.nan
            df["shorts_win_after"] = np.nan
            df["longs_win_after"] = np.nan
            df["dual_loss"] = 0
            df["entered_before"] = np.nan

            for i in range(len(df)):
                if df["entry"][i]:
                    long_target = df["entry"][i] * (1 + win_perc / 100)
                    short_target = df["entry"][i] * (1 - win_perc / 100)
                    long_stop_loss = df["entry"][i] * (1 - loss_perc / 100)
                    short_stop_loss = df["entry"][i] * (1 + loss_perc / 100)
                    df.loc[i, 'long_target'] = long_target
                    df.loc[i, 'long_stop_loss'] = long_stop_loss
                    for j in range(i, len(df)):
                        if df["high"][j] >= long_target:
                            if df["low"][j] <= long_stop_loss:
                                df.loc[i, 'if_long'] = -1
                                df.loc[i, 'dual_loss'] = 1
                                df.loc[i, 'entered_before'] = j - i
                            else:
                                df.loc[i, 'if_long'] = 1
                                df.loc[i, 'longs_win_after'] = j - i
                            break
                        elif df["low"][j] <= long_stop_loss:
                            df.loc[i, 'if_long'] = -1
                            break
                    df.loc[i, 'short_target'] = short_target
                    df.loc[i, 'short_stop_loss'] = short_stop_loss
                    for j in range(i, len(df)):
                        if df["low"][j] <= short_target:
                            if df["high"][j] >= short_stop_loss:
                                df.loc[i, 'if_short'] = -1
                                df.loc[i, 'dual_loss'] = 1
                                df.loc[i, 'entered_before'] = j - i
                            else:
                                df.loc[i, 'if_short'] = 1
                                df.loc[i, 'shorts_win_after'] = j - i
                            break
                        elif df["high"][j] >= short_stop_loss:
                            df.loc[i, 'if_short'] = -1
                            break
            # Save the updated dataframe to the CSV file
            df.to_csv(file_path, index=False)

    return ("calculated wins and losses ")

In [ ]:
def calculate_indicator_values_new(master_dictionary, monthly_or_daily_1_2, win_perc, loss_perc):
    # Iterate over the symbols and chart times
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            # Construct the file name
            file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
            file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

            # Read the CSV file into a dataframe
            df = pd.read_csv(file_path)
            print(df.dtypes)
            #########Overlap Studies
            new_columns = pd.DataFrame()
            new_columns['HT_TRENDLINE'] = talib.HT_TRENDLINE(df['close'])
            #         df['MAMA'], df['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
            #         df['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
            new_columns['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0, maximum=0)
            new_columns['SAREXT'] = talib.SAREXT(df['high'], df['low'])
            new_columns['T3'] = talib.T3(df['close'], timeperiod=5, vfactor=0)
#             #########Momentum Indicators
            new_columns['APO'] = talib.APO(df['close'], fastperiod=12, slowperiod=26)
            new_columns['BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
            macd, macd_signal, macd_hist = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
            new_columns['MACD'] = macd
            new_columns['MACD_signal'] = macd_signal
            new_columns['MACD_hist'] = macd_hist
            new_columns['PPO'] = talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
            new_columns['TRIX'] = talib.TRIX(df['close'])
            new_columns['ULTOSC'] = talib.ULTOSC(df['high'], df['low'], df['close'])
            new_columns['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'])

            

            # Not Working ATM
            #         new_columns['STOCH'] = talib.STOCH(df['high'], df['low'], df['close'])
            #         new_columns['STOCHF'] = talib.STOCHF(df['high'], df['low'], df['close'])
            #         new_columns['STOCHRSI'] = talib.STOCHRSI(df['close'])
            #         new_columns['MACDEXT'] = talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
            #         new_columns['MACDFIX'] = talib.MACDFIX(df['close'], signalperiod=9)
            #########Volume Indicators
            new_columns['AD'] = talib.AD(df['high'], df['low'], df['close'], df['volume'])
            new_columns['ADOSC'] = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
            new_columns['OBV'] = talib.OBV(df['close'], df['volume'])

            #########Cycle Indicators
            new_columns['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close'])
            new_columns['HT_DCPHASE'] = talib.HT_DCPHASE(df['close'])
            new_columns['HT_PHASOR_inphase'], new_columns['HT_PHASOR_quadrature'] = talib.HT_PHASOR(df['close'])
            # new_columns['HT_SINE'] = talib.HT_SINE(df['close'])
            new_columns['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close'])

            #########Price Transform
            new_columns['AVGPRICE'] = talib.AVGPRICE(df['open'], df['high'], df['low'], df['close'])
            new_columns['MEDPRICE'] = talib.MEDPRICE(df['high'], df['low'])
            new_columns['TYPPRICE'] = talib.TYPPRICE(df['high'], df['low'], df['close'])
            new_columns['WCLPRICE'] = talib.WCLPRICE(df['high'], df['low'], df['close'])
            #########Volatility Indicators
            new_columns['TRANGE'] = talib.TRANGE(df['high'], df['low'], df['close'])
            #########Pattern Recognition
            new_columns['CDL2CROWS'] = talib.CDL2CROWS(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDL3BLACKCROWS'] = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDL3INSIDE'] = talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDL3LINESTRIKE'] = talib.CDL3LINESTRIKE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDL3OUTSIDE'] = talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDL3STARSINSOUTH'] = talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLABANDONEDBABY'] = talib.CDLABANDONEDBABY(df['open'], df['high'], df['low'], df['close'], penetration=0)

            new_columns['CDLADVANCEBLOCK'] = talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLBELTHOLD'] = talib.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLBREAKAWAY'] = talib.CDLBREAKAWAY(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLCLOSINGMARUBOZU'] = talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLCONCEALBABYSWALL'] = talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLCOUNTERATTACK'] = talib.CDLCOUNTERATTACK(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLDARKCLOUDCOVER'] = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'], penetration=0)

            new_columns['CDLDOJI'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLDOJISTAR'] = talib.CDLDOJISTAR(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLDRAGONFLYDOJI'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLENGULFING'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLEVENINGDOJISTAR'] = talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)

            new_columns['CDLEVENINGSTAR'] = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
            new_columns['CDLGAPSIDESIDEWHITE'] = talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLGRAVESTONEDOJI'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHAMMER'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHANGINGMAN'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHARAMI'] = talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHARAMICROSS'] = talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHIGHWAVE'] = talib.CDLHIGHWAVE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHIKKAKE'] = talib.CDLHIKKAKE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHIKKAKEMOD'] = talib.CDLHIKKAKEMOD(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLHOMINGPIGEON'] = talib.CDLHOMINGPIGEON(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLIDENTICAL3CROWS'] = talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLINNECK'] = talib.CDLINNECK(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLINVERTEDHAMMER'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLKICKING'] = talib.CDLKICKING(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLKICKINGBYLENGTH'] = talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLLADDERBOTTOM'] = talib.CDLLADDERBOTTOM(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLLONGLEGGEDDOJI'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLLONGLINE'] = talib.CDLLONGLINE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLMARUBOZU'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLMATCHINGLOW'] = talib.CDLMATCHINGLOW(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLMATHOLD'] = talib.CDLMATHOLD(df['open'], df['high'], df['low'], df['close'], penetration=0)
            new_columns['CDLMORNINGDOJISTAR'] = talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLMORNINGSTAR'] = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLONNECK'] = talib.CDLONNECK(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLPIERCING'] = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLRICKSHAWMAN'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLRISEFALL3METHODS'] = talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLSEPARATINGLINES'] = talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLSHOOTINGSTAR'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLSHORTLINE'] = talib.CDLSHORTLINE(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLSPINNINGTOP'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLSTALLEDPATTERN'] = talib.CDLSTALLEDPATTERN(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLSTICKSANDWICH'] = talib.CDLSTICKSANDWICH(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLTAKURI'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLTASUKIGAP'] = talib.CDLTASUKIGAP(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLTHRUSTING'] = talib.CDLTHRUSTING(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLTRISTAR'] = talib.CDLTRISTAR(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLUNIQUE3RIVER'] = talib.CDLUNIQUE3RIVER(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLUPSIDEGAP2CROWS'] = talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'], df['close'])
            new_columns['CDLXSIDEGAP3METHODS'] = talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'], df['close'])
            #########Statistic Functions
            new_columns['LINEARREG'] = talib.LINEARREG(df['close'])
            new_columns['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['close'])
            new_columns['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['close'])
            new_columns['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['close'])
            #         new_columns['STDDEV'] = df['close'].rolling(timeperiod).std()
            new_columns['TSF'] = talib.TSF(df['close'])
            new_columns['VAR'] = talib.VAR(df['close'])
            # Iterate over the time periods
            for timeperiod in master_dictionary["timeperiods"]:
                #########Overlap Studies
                new_columns[f'BB_upper_{timeperiod}'], new_columns[f'BB_middle_{timeperiod}'], new_columns[
                    f'BB_lower_{timeperiod}'] = talib.BBANDS(df['close'], timeperiod=timeperiod)
                new_columns[f'DEMA_{timeperiod}'] = talib.DEMA(df['close'], timeperiod=timeperiod)
                new_columns[f'EMA_{timeperiod}'] = talib.EMA(df['close'], timeperiod=timeperiod)
                new_columns[f'KAMA_{timeperiod}'] = talib.KAMA(df['close'], timeperiod=timeperiod)
                new_columns[f'MA_{timeperiod}'] = talib.MA(df['close'], timeperiod=timeperiod)
                #         new_columns['MAMA'], new_columns['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
                #         new_columns['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
                new_columns[f'MIDPOINT_{timeperiod}'] = talib.MIDPOINT(df['close'], timeperiod=timeperiod)
                new_columns[f'MIDPRICE_{timeperiod}'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=timeperiod)
                new_columns[f'SMA_{timeperiod}'] = talib.SMA(df['close'], timeperiod=timeperiod)
                new_columns[f'TEMA_{timeperiod}'] = talib.TEMA(df['close'], timeperiod=timeperiod)
                new_columns[f'TRIMA_{timeperiod}'] = talib.TRIMA(df['close'], timeperiod=timeperiod)
                new_columns[f'WMA_{timeperiod}'] = talib.WMA(df['close'], timeperiod=timeperiod)
                new_columns[f'ADX_{timeperiod}'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                new_columns[f'ADXR_{timeperiod}'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
                                                                                           timeperiod=timeperiod)
                new_columns[f'AROONOSC_{timeperiod}'] = talib.AROONOSC(df['high'], df['low'], timeperiod=timeperiod)
                new_columns[f'CCI_{timeperiod}'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                new_columns[f'CMO_{timeperiod}'] = talib.CMO(df['close'], timeperiod=timeperiod)
                new_columns[f'DX_{timeperiod}'] = talib.DX(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                new_columns[f'MFI_{timeperiod}'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'],
                                                    timeperiod=timeperiod)
                new_columns[f'MINUS_DI_{timeperiod}'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                new_columns[f'MINUS_DM_{timeperiod}'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=timeperiod)
                new_columns[f'MOM_{timeperiod}'] = talib.MOM(df['close'], timeperiod=timeperiod)
                new_columns[f'PLUS_DI_{timeperiod}'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                new_columns[f'PLUS_DM_{timeperiod}'] = talib.PLUS_DM(df['high'], df['low'], timeperiod=timeperiod)
                new_columns[f'ROC_{timeperiod}'] = talib.ROC(df['close'], timeperiod=timeperiod)
                new_columns[f'ROCP_{timeperiod}'] = talib.ROCP(df['close'], timeperiod=timeperiod)
                new_columns[f'ROCR_{timeperiod}'] = talib.ROCR(df['close'], timeperiod=timeperiod)
                new_columns[f'ROCR100_{timeperiod}'] = talib.ROCR100(df['close'], timeperiod=timeperiod)
                new_columns[f'RSI_{timeperiod}'] = talib.RSI(df['close'], timeperiod=timeperiod)

                new_columns[f'ATR_{timeperiod}'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                new_columns[f'NATR_{timeperiod}'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                #########Statistic Functions
                new_columns[f'BETA_{timeperiod}'] = talib.BETA(df['high'], df['low'], timeperiod=timeperiod)
                new_columns[f'CORREL_{timeperiod}'] = talib.CORREL(df['high'], df['low'], timeperiod=timeperiod)
            # Save the updated dataframe to the CSV file
            df = pd.concat([df, new_columns], axis=1)
            df.to_csv(file_path, index=False)
    return ("indicators are added to the csv")

In [ ]:
# def calculate_indicator_values_old(master_dictionary, monthly_or_daily_1_2):
#     # Iterate over the symbols and chart times
#     for symbol in master_dictionary["symbols"]:
#         for chart_time in master_dictionary["chart_times"]:
#             # Construct the file name
#             file_name = f"{symbol}-{chart_time}.csv"
#             if monthly_or_daily_1_2 in ["m", "M", "monthly", 1]:
#                 # Construct the file path
#                 file_path = Path(output_dir) / f"{symbol}-{chart_time}-monthly_data/{file_name}"
#             elif monthly_or_daily_1_2 in ["d", "D", "daily", 2]:
#                 # Construct the file path
#                 file_path = Path(output_dir) / f"{symbol}-{chart_time}-daily_data/{file_name}"
#             # Read the CSV file into a dataframe
#             df = pd.read_csv(file_path)
#             print(df.dtypes)
#             #########Overlap Studies
#             df['BB_upper'], df['BB_middle'], df['BB_lower'] = talib.BBANDS(df['close'], timeperiod=5)
#             df['DEMA'] = talib.DEMA(df['close'], timeperiod=30)
#             df['EMA-50'] = talib.EMA(df['close'], timeperiod=50)
#             df['EMA-200'] = talib.EMA(df['close'], timeperiod=200)
#             df['HT_TRENDLINE'] = talib.HT_TRENDLINE(df['close'])
#             df['KAMA'] = talib.KAMA(df['close'], timeperiod=30)
#             df['MA'] = talib.MA(df['close'], timeperiod=14)
#             #         df['MAMA'], df['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
#             #         df['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
#             df['MIDPOINT'] = talib.MIDPOINT(df['close'], timeperiod=14)
#             df['MIDPRICE'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=14)
#             df['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0, maximum=0)
#             df['SAREXT'] = talib.SAREXT(df['high'], df['low'])
#             df['SMA'] = talib.SMA(df['close'], timeperiod=14)
#             df['T3'] = talib.T3(df['close'], timeperiod=5, vfactor=0)
#             df['TEMA'] = talib.TEMA(df['close'])
#             df['TRIMA'] = talib.TRIMA(df['close'])
#             df['WMA'] = talib.WMA(df['close'])
#             #########Momentum Indicators
#             df['ADX'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
#             df['ADXR'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=14)
#             df['APO'] = talib.APO(df['close'], fastperiod=12, slowperiod=26)
#             df['AROON_up'], df['AROON_down'] = talib.AROON(df['high'], df['low'], timeperiod=14)
#             df['AROONOSC'] = talib.AROONOSC(df['high'], df['low'], timeperiod=14)
#             df['BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
#             df['CCI'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
#             df['CMO'] = talib.CMO(df['close'], timeperiod=14)
#             df['DX'] = talib.DX(df['high'], df['low'], df['close'], timeperiod=14)
#             df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26,
#                                                                         signalperiod=9)
#             df['MFI'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=14)
#             df['MINUS_DI'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
#             df['MINUS_DM'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=14)
#             df['MOM'] = talib.MOM(df['close'], timeperiod=14)
#             df['PLUS_DI'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
#             df['PLUS_DM'] = talib.PLUS_DM(df['high'], df['low'], timeperiod=14)
#             df['PPO'] = talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
#             df['ROC'] = talib.ROC(df['close'], timeperiod=14)
#             df['ROCP'] = talib.ROCP(df['close'], timeperiod=14)
#             df['ROCR'] = talib.ROCR(df['close'], timeperiod=14)
#             df['ROCR100'] = talib.ROCR100(df['close'], timeperiod=14)
#             df['RSI'] = talib.RSI(df['close'], timeperiod=8)
#             df['TRIX'] = talib.TRIX(df['close'])
#             df['ULTOSC'] = talib.ULTOSC(df['high'], df['low'], df['close'])
#             df['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'])

#             # Not Working ATM
#             #         df['STOCH'] = talib.STOCH(df['high'], df['low'], df['close'])
#             #         df['STOCHF'] = talib.STOCHF(df['high'], df['low'], df['close'])
#             #         df['STOCHRSI'] = talib.STOCHRSI(df['close'])
#             #         df['MACDEXT'] = talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
#             #         df['MACDFIX'] = talib.MACDFIX(df['close'], signalperiod=9)
#             #########Volume Indicators
#             df['AD'] = talib.AD(df['high'], df['low'], df['close'], df['volume'])
#             df['ADOSC'] = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
#             df['OBV'] = talib.OBV(df['close'], df['volume'])
#             #########Cycle Indicators
#             df['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close'])
#             df['HT_DCPHASE'] = talib.HT_DCPHASE(df['close'])
#             df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(df['close'])
#             #         df['HT_SINE'] = talib.HT_SINE(df['close'])
#             df['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close'])
#             #########Price Transform
#             df['AVGPRICE'] = talib.AVGPRICE(df['open'], df['high'], df['low'], df['close'])
#             df['MEDPRICE'] = talib.MEDPRICE(df['high'], df['low'])
#             df['TYPPRICE'] = talib.TYPPRICE(df['high'], df['low'], df['close'])
#             df['WCLPRICE'] = talib.WCLPRICE(df['high'], df['low'], df['close'])
#             #########Volatility Indicators
#             df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
#             df['NATR'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=14)
#             df['TRANGE'] = talib.TRANGE(df['high'], df['low'], df['close'])
#             #########Pattern Recognition
#             df['CDL2CROWS'] = talib.CDL2CROWS(df['open'], df['high'], df['low'], df['close'])
#             df['CDL3BLACKCROWS'] = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
#             df['CDL3INSIDE'] = talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close'])
#             df['CDL3LINESTRIKE'] = talib.CDL3LINESTRIKE(df['open'], df['high'], df['low'], df['close'])
#             df['CDL3OUTSIDE'] = talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
#             df['CDL3STARSINSOUTH'] = talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close'])
#             df['CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])
#             df['CDLABANDONEDBABY'] = talib.CDLABANDONEDBABY(df['open'], df['high'], df['low'], df['close'],
#                                                             penetration=0)
#             df['CDLADVANCEBLOCK'] = talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close'])
#             df['CDLBELTHOLD'] = talib.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close'])
#             df['CDLBREAKAWAY'] = talib.CDLBREAKAWAY(df['open'], df['high'], df['low'], df['close'])
#             df['CDLCLOSINGMARUBOZU'] = talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close'])
#             df['CDLCONCEALBABYSWALL'] = talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'], df['close'])
#             df['CDLCOUNTERATTACK'] = talib.CDLCOUNTERATTACK(df['open'], df['high'], df['low'], df['close'])
#             df['CDLDARKCLOUDCOVER'] = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'],
#                                                               penetration=0)
#             df['CDLDOJI'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
#             df['CDLDOJISTAR'] = talib.CDLDOJISTAR(df['open'], df['high'], df['low'], df['close'])
#             df['CDLDRAGONFLYDOJI'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
#             df['CDLENGULFING'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
#             df['CDLEVENINGDOJISTAR'] = talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'],
#                                                                 penetration=0)
#             df['CDLEVENINGSTAR'] = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
#             df['CDLGAPSIDESIDEWHITE'] = talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'], df['close'])
#             df['CDLGRAVESTONEDOJI'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHAMMER'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHANGINGMAN'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHARAMI'] = talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHARAMICROSS'] = talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHIGHWAVE'] = talib.CDLHIGHWAVE(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHIKKAKE'] = talib.CDLHIKKAKE(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHIKKAKEMOD'] = talib.CDLHIKKAKEMOD(df['open'], df['high'], df['low'], df['close'])
#             df['CDLHOMINGPIGEON'] = talib.CDLHOMINGPIGEON(df['open'], df['high'], df['low'], df['close'])
#             df['CDLIDENTICAL3CROWS'] = talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'], df['close'])
#             df['CDLINNECK'] = talib.CDLINNECK(df['open'], df['high'], df['low'], df['close'])
#             df['CDLINVERTEDHAMMER'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
#             df['CDLKICKING'] = talib.CDLKICKING(df['open'], df['high'], df['low'], df['close'])
#             df['CDLKICKINGBYLENGTH'] = talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'], df['close'])
#             df['CDLLADDERBOTTOM'] = talib.CDLLADDERBOTTOM(df['open'], df['high'], df['low'], df['close'])
#             df['CDLLONGLEGGEDDOJI'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
#             df['CDLLONGLINE'] = talib.CDLLONGLINE(df['open'], df['high'], df['low'], df['close'])
#             df['CDLMARUBOZU'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
#             df['CDLMATCHINGLOW'] = talib.CDLMATCHINGLOW(df['open'], df['high'], df['low'], df['close'])
#             df['CDLMATHOLD'] = talib.CDLMATHOLD(df['open'], df['high'], df['low'], df['close'], penetration=0)
#             df['CDLMORNINGDOJISTAR'] = talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
#             df['CDLMORNINGSTAR'] = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
#             df['CDLONNECK'] = talib.CDLONNECK(df['open'], df['high'], df['low'], df['close'])
#             df['CDLPIERCING'] = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
#             df['CDLRICKSHAWMAN'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
#             df['CDLRISEFALL3METHODS'] = talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close'])
#             df['CDLSEPARATINGLINES'] = talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'], df['close'])
#             df['CDLSHOOTINGSTAR'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
#             df['CDLSHORTLINE'] = talib.CDLSHORTLINE(df['open'], df['high'], df['low'], df['close'])
#             df['CDLSPINNINGTOP'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
#             df['CDLSTALLEDPATTERN'] = talib.CDLSTALLEDPATTERN(df['open'], df['high'], df['low'], df['close'])
#             df['CDLSTICKSANDWICH'] = talib.CDLSTICKSANDWICH(df['open'], df['high'], df['low'], df['close'])
#             df['CDLTAKURI'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
#             df['CDLTASUKIGAP'] = talib.CDLTASUKIGAP(df['open'], df['high'], df['low'], df['close'])
#             df['CDLTHRUSTING'] = talib.CDLTHRUSTING(df['open'], df['high'], df['low'], df['close'])
#             df['CDLTRISTAR'] = talib.CDLTRISTAR(df['open'], df['high'], df['low'], df['close'])
#             df['CDLUNIQUE3RIVER'] = talib.CDLUNIQUE3RIVER(df['open'], df['high'], df['low'], df['close'])
#             df['CDLUPSIDEGAP2CROWS'] = talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'], df['close'])
#             df['CDLXSIDEGAP3METHODS'] = talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'], df['close'])
#             #########Statistic Functions
#             df['BETA'] = talib.BETA(df['high'], df['low'], timeperiod=5)
#             df['CORREL'] = talib.CORREL(df['high'], df['low'], timeperiod=30)
#             df['LINEARREG'] = talib.LINEARREG(df['close'])
#             df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['close'])
#             df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['close'])
#             df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['close'])
#             #         df['STDDEV'] = df['close'].rolling(timeperiod).std()
#             df['TSF'] = talib.TSF(df['close'])
#             df['VAR'] = talib.VAR(df['close'])

#             # Save the updated dataframe to the CSV file
#             df.to_csv(file_path, index=False)
#     return ("indicators are added to the csv")

In [ ]:
print("data creation utilities successfully initialized")
